# Linear Model with TensorFlow

Let's use the MTA Subway data, combined with weather data, to predict the number of entries at the station.
In this notebook I will use TensorFlow to create a simple linear regression model.

Start with importing necessary libraries and loading data.

In [4]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from exploratory_analysis import *

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

turnstile, weather = load_data('master_turnstile_file.txt', 'weather_underground.csv')
data = merge_turnstile_weather(turnstile, weather)

data.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,EXITSn_hourly,ENTRIESn_hourly,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
0,A002,R051,02-00-00,2011-05-21,00:00:00,REGULAR,3169391,1097585,0.0,0.0,...,0.1,0.0,41,1.8,0.0,0,81,67,16,19
1,A002,R051,02-00-00,2011-05-21,04:00:00,REGULAR,3169415,1097588,3.0,24.0,...,0.1,0.0,41,1.8,0.0,0,81,67,16,19
2,A002,R051,02-00-00,2011-05-21,08:00:00,REGULAR,3169431,1097607,19.0,16.0,...,0.1,0.0,41,1.8,0.0,0,81,67,16,19
3,A002,R051,02-00-00,2011-05-21,12:00:00,REGULAR,3169506,1097686,79.0,75.0,...,0.1,0.0,41,1.8,0.0,0,81,67,16,19
4,A002,R051,02-00-00,2011-05-21,16:00:00,REGULAR,3169693,1097734,48.0,187.0,...,0.1,0.0,41,1.8,0.0,0,81,67,16,19
